In [1]:
from sksurv.preprocessing import OneHotEncoder
from sklearn.externals import joblib
import pickle
import pandas as pd
import numpy as np

def survivalTable(modelName, raw_data):
    '''
    Calculate survival rate in years of interest
    '''
    OHE_LOCATION = "C:\\SMU_v2\\OHE\\"  
    interval = list([0.5,1,2,5,10])

    for k,v in raw_data.items():
        if str(v[0]).isalpha():
            raw_data[k] = v[0].lower()
        
    raw_data = pd.DataFrame.from_dict(raw_data)
    
    model = joblib.load('Model_folder\\{}.pkl'.format(modelName))

    with open( "{}{}{}".format(OHE_LOCATION, modelName[:-4], '_encoder.pickle'), 'rb') as f:
            enc = pickle.load(f) 
        
    #type case object to category
    typeCastList = list(raw_data.select_dtypes(include=[object]).columns)
    raw_data[typeCastList] = raw_data[typeCastList].astype("category")
    data = enc.transform(raw_data)

    surv = model.predict_survival_function(data)
    
    dic = {}
    
    for i, s in enumerate(surv):
        x = model.event_times_
        y = s
    graphaxis = pd.DataFrame({'x':x,'y':y}, columns = ['x','y'])
    for i in interval:
        result = np.where(x > (365.25*(i+1)))[0][0]
        dic[i] = y[result]

    return dic,graphaxis

C:\Users\LINGXING\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
def loadOHE(df,OHE_LOCATION = "C:\\SMU_v2\\OHE\\", name=""):
    '''
    load enconder to OHE new raw data for prediction
    '''
    with open( "{}{}{}".format(OHE_LOCATION, name, '_encoder.pickle'), 'rb') as f:
        enc = pickle.load(f) 
    
    #type case object to category
    typeCastList = list(df.select_dtypes(include=[object]).columns)
    df[typeCastList] = df[typeCastList].astype("category")
    OHE_New_Data = enc.transform(df)
    
    return OHE_New_Data

In [10]:
# group 1
stage = "dcis/non-invasive"

if stage == "dcis/non-invasive":
    raw_data = {
            'ER': ['positive'],\
            'PR': ['positive'],\
            'Her2': ['negative'],\
            'size_precise': [1.3],\
            'nodespos': [0],\
            'Age_@_Dx': [21],\
            'diff': ['grade 3']
           }

# group 3
elif stage == "4":
    raw_data = {
        'ER': ['positive'],\
        'PR': ['positive'],\
        'Her2': ['negative'],\
        'size_precise': [1.3],\
        'nodespos': [0],\
        'Age_@_Dx': [21],\
        'T':['Tis'],\
        'N': ['n0'],\
        'M': ['m0']
    }

# group 2
else:
    raw_data = {
        'ER': ['positive'],\
        'PR': ['positive'],\
        'Her2': ['negative'],\
        'Size': [">"],\
        'Age_@_Dx': [21],\
        'diff': ['m0']
    }
    
raw_data = pd.DataFrame.from_dict(raw_data)
raw_data = loadOHE(raw_data,OHE_LOCATION = "C:\\SMU_v2\\OHE\\", name="group 1_layer 4")
raw_data
# z,DF = survivalTable('group 1_layer 4_rsf',raw_data)
# print(z)
# DF

,Age_@_Dx,diff=grade 2,diff=grade 3,diff=unknown,ER=negative,ER=positive,ER=unknown,PR=negative,PR=positive,PR=unknown,Her2=negative,Her2=not done,Her2=positive,Her2=unknown,size_precise,nodespos
0,21,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.3,0
